## - Import

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch
import numpy as np 
import pandas as pd  
import os 
from torchvision.datasets import SVHN 
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from tqdm import tqdm


import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchvision.utils import save_image

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
torch.cuda.empty_cache()

## - Parameters

In [ ]:
transform = transforms.Compose([
        # transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,))
])

In [23]:
train_dataset=SVHN(split='train',root='data/train',transform=transform)
# test_dataset=SVHN(split='test',root='data/test',transform=transform)

vars(train_dataset)

{'root': 'data/train',
 'transform': Compose(
     ToTensor()
     Normalize(mean=(0.5,), std=(0.5,))
 ),
 'target_transform': None,
 'transforms': StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5,), std=(0.5,))
            ),
 'split': 'train',
 'url': 'http://ufldl.stanford.edu/housenumbers/train_32x32.mat',
 'filename': 'train_32x32.mat',
 'file_md5': 'e26dedcc434d2e4c54c9b2d4a06d8373',
 'data': array([[[[ 33,  15,  15, ...,  72,  82,  89],
          [ 28,  14,  14, ...,  40,  67,  83],
          [ 40,  18,  16, ...,  23,  35,  72],
          ...,
          [ 86,  87,  82, ..., 104, 104, 103],
          [ 84,  86,  82, ..., 110, 106, 106],
          [ 85,  89,  85, ..., 111, 109, 103]],
 
         [[ 30,  23,  17, ...,  65,  77,  79],
          [ 39,  25,  20, ...,  39,  58,  78],
          [ 41,  21,  17, ...,  29,  42,  69],
          ...,
          [ 81,  82,  79, ..., 104, 105, 105],
          [ 86,  79,  76, ..., 103, 105, 10

In [ ]:
class_list = {'0','1','2','3','4','5','6','7','8','9'}

In [ ]:
# Data


img_size = 32 # Image size
batch_size = 8  # Batch size

# Model
z_size = 100
generator_layer_size = [256, 512, 1024]
discriminator_layer_size = [1024, 512, 256]

# Training
epochs = 30  # Train epochs
learning_rate = 1e-4
class_num=10
channels=3

In [ ]:
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
# val_loader = DataLoader(test_dataset, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
a,b=next(iter(train_loader))
a.shape

## - Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, generator_layer_size, z_size, img_size, class_num):
        super().__init__()
        
        self.z_size = z_size
        self.img_size = img_size
        
        self.label_emb = nn.Embedding(class_num, class_num)
        
        self.model = nn.Sequential(
            nn.Linear(self.z_size + class_num, generator_layer_size[0]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(generator_layer_size[0], generator_layer_size[1]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(generator_layer_size[1], generator_layer_size[2]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(generator_layer_size[2], self.img_size * self.img_size*channels),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        
        # Reshape z
        z = z.view(-1, self.z_size)
        
        # One-hot vector to embedding vector
        c = self.label_emb(labels)
        
        # Concat image & label
        x = torch.cat([z, c], 1)
        # print(x.shape)
        # Generator out
        out = self.model(x)
        
        return out.view(-1,channels, self.img_size, self.img_size)

## - Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, discriminator_layer_size, img_size, class_num):
        super().__init__()
        
        self.label_emb = nn.Embedding(class_num, class_num)
        self.img_size = img_size
        
        self.model = nn.Sequential(
            nn.Linear(self.img_size * self.img_size*channels + class_num, discriminator_layer_size[0]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(discriminator_layer_size[0], discriminator_layer_size[1]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(discriminator_layer_size[1], discriminator_layer_size[2]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(discriminator_layer_size[2], 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        
        # Reshape fake image
        x = x.view(-1, channels*self.img_size * self.img_size)
        # print(x.shape)
        # One-hot vector to embedding vector
        c = self.label_emb(labels)
        # print(c.shape)
        
        # Concat image & label
        x = torch.cat([x, c], 1)
        
        # Discriminator out
        out = self.model(x)
        
        return out.squeeze()

In [ ]:
# Define generator
generator = Generator(generator_layer_size, z_size, img_size, class_num).to(device)
# Define discriminator
discriminator = Discriminator(discriminator_layer_size, img_size, class_num).to(device)

## - Adversarial Learning of Generator & Discriminator

In [ ]:
# Loss function
criterion = nn.BCELoss()

In [ ]:
# Optimizer
g_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    
    # Init gradient
    g_optimizer.zero_grad()
    
    # Building z
    z = Variable(torch.randn(batch_size, z_size)).to(device)
    
    # Building fake labels
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, class_num, batch_size))).to(device)
    
    # Generating fake images
    fake_images = generator(z, fake_labels)
    
    # Disciminating fake images
    validity = discriminator(fake_images, fake_labels)
    
    # Calculating discrimination loss (fake images)
    g_loss = criterion(validity, Variable(torch.ones(batch_size)).to(device))
    
    # Backword propagation
    g_loss.backward()
    
    #  Optimizing generator
    g_optimizer.step()
    
    return g_loss.data

In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    
    # Init gradient 
    d_optimizer.zero_grad()

    # Disciminating real images
    real_validity = discriminator(real_images, labels)
    
    # Calculating discrimination loss (real images)
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)).to(device))
    
    # Building z
    z = Variable(torch.randn(batch_size, z_size)).to(device)
    
    # Building fake labels
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, class_num, batch_size))).to(device)
    
    # Generating fake images
    fake_images = generator(z, fake_labels)
    
    # Disciminating fake images
    fake_validity = discriminator(fake_images, fake_labels)
    
    # Calculating discrimination loss (fake images)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)).to(device))
    
    # Sum two losses
    d_loss = real_loss + fake_loss
    
    # Backword propagation
    d_loss.backward()
    
    # Optimizing discriminator
    d_optimizer.step()
    
    return d_loss.data

In [ ]:
def train_gan(epochs,discriminator,train_loader,generator,g_optimizer=g_optimizer,batch_size=batch_size,d_optimizer=d_optimizer):
    for epoch in range(epochs):
        
        print('Starting epoch {}...'.format(epoch+1))
        
        for i, (images, labels) in enumerate(train_loader):
            
            # Train data
            real_images = Variable(images).to(device)
            labels = Variable(labels).to(device)
            
            # Set generator train
            generator.train()
            
            # Train discriminator
            d_loss = discriminator_train_step(len(real_images), discriminator,
                                            generator, d_optimizer, criterion,
                                            real_images, labels)
            
            # Train generator
            g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)
        
        # Set generator eval
        generator.eval()
        
        print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))
        
        # Building z 
        z = Variable(torch.randn(class_num-1, z_size)).to(device)
        
        # Labels 0 ~ 8
        labels = Variable(torch.LongTensor(np.arange(class_num-1))).to(device)
        
        # Generating images
        sample_images = generator(z, labels).data.cpu()
        print(sample_images.shape)
        # Show images
        grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
        plt.imshow(grid)
        plt.show()

In [ ]:
train_gan(epochs=epochs,generator=generator,discriminator=discriminator,train_loader=train_loader)

## - Show Generating Images

In [ ]:
# Building z 
z = Variable(torch.randn(z_size, z_size)).to(device)

# Labels 0 ~ 9
labels = Variable(torch.LongTensor([i for _ in range(class_num*2) for i in range(class_num)])).to(device)

# Generating images
sample_images = generator(z, labels).data.cpu()

# Show images
grid = make_grid(sample_images, nrow=class_num, normalize=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(grid)
_ = plt.yticks([])
_ = plt.xticks(np.arange(15, 300, 30), class_list, rotation=45, fontsize=20)

In [ ]:
torch.randn(2, 3)